In [2]:
"""Extraction Python

This module creates functions to ingest reports/'%YEAR-YEAR%/'%YR-%YR Scorecard and Jurisdiction Break Out.xlsx'

Example:
        The functions are intended to be used in the Main.py file of this project
        

Section breaks are created by resuming unindented text. Section breaks
are also implicitly created anytime a new section starts.

Attributes:
    module_level_variable1 (int): Module level variables may be documented in
        either the ``Attributes`` section of the module docstring, or in an
        inline docstring immediately following the variable.

        Either form is acceptable, but the two should not be mixed. Choose
        one convention to document module level variables and be consistent
        with it.

Todo:
    * For module TODOs
    * You have to also use ``sphinx.ext.todo`` extension

.. _Google Python Style Guide:
   http://google.github.io/styleguide/pyguide.html

"""

"Extraction Python\n\nThis module creates functions to ingest reports/'%YEAR-YEAR%/'%YR-%YR Scorecard and Jurisdiction Break Out.xlsx'\n\nExample:\n        The functions are intended to be used in the Main.py file of this project\n        \n\nSection breaks are created by resuming unindented text. Section breaks\nare also implicitly created anytime a new section starts.\n\nAttributes:\n    module_level_variable1 (int): Module level variables may be documented in\n        either the ``Attributes`` section of the module docstring, or in an\n        inline docstring immediately following the variable.\n\n        Either form is acceptable, but the two should not be mixed. Choose\n        one convention to document module level variables and be consistent\n        with it.\n\nTodo:\n    * For module TODOs\n    * You have to also use ``sphinx.ext.todo`` extension\n\n.. _Google Python Style Guide:\n   http://google.github.io/styleguide/pyguide.html\n\n"

In [8]:
import os
import pandas as pd

# Data locations, Move to config file
source_location = os.path.join('..','..','data','external')
target_location = os.path.join('..','..','data','interim')

# To External Config File
source_file_name              = '2016-17 Scorecard and Jurisdiction Break Out.xlsx'
target_kpi_file_name          = 'scorecard_extracted_kpi.csv'
target_metrics_file_name      = 'scorecard_extracted_metrics.csv'
target_jurisdiction_file_name = 'scorecard_extracted_juris_metrics.csv'

# Source data File Path
source_file_path = os.path.join(source_location, source_file_name)

# KPI & Metrics File Paths
target_kpi_file_path          = os.path.join(target_location, target_kpi_file_name)
target_metrics_file_path      = os.path.join(target_location, target_metrics_file_name)
target_jurisdiction_file_path = os.path.join(target_location, target_jurisdiction_file_name)

# Potentially to External Config File

##########################################################################
# Data & Structure I am going to need to reference later in order to pull out relevant metrics

# In the future I should't have to do this. I can use pd.ExcelFile() to read in all sheets and parse them all
# and being cleverly selective with what fields I choose to grab.... May be a bit more robust, however these Scorecards are sacred and unlikely to change
# in terms of sheet names

# KPI metrics sheets
kpi_metrics_sheets = ['July',
'August',
'September',
'October',
'November',
'December',
'January',
'February',
'March',
'April',
'May',
'June']

# All Metrics Sheets
metrics_sheets = ['July Metrics',
'August Metrics',
'September Metrics',
'October Metrics',
'November Metrics',
'December Metrics',
'January Metrics',
'February Metrics',
'March Metrics',
'April Metrics',
'May Metrics',
'June Metrics']

# KPI Metrics to be pulled from rows on KPI Metrics Sheets
kpi_metrics = ['Group Room Nights Booked',
              'Pace Report Growth*',
              'Communication Effectiveness',
              'Brand Preference Index',
              'Targeted Earned Media',
              'Website Visits',
              'Social Engagements',
              '% of Visitors Influenced to stay longer']

# Metrics to be pulled from rows on Metrics Sheets
metrics = ['Definite Group Bookings',
          'Group Leads',
          'Group Lead Room Nights',
          'Group Room Night Index',
          'Comp set Rev Par growth',
          'Unaided Brand Awareness',
          'Total Earned media (all segments)',
          'PR Impressions',
          'Journalists Assisted',
          'Facebook Fans',
          'Twitter Followers',
          'Instagram Followers',
          'Visitor Database',
          'Room Nights (RN) Booked through Web',
          'Economic Impact of RN Booked through Web',
           'Total Onsite Referrals (clicks to a member listing page)',
           'Total Offsite Referrals (clicks to a direct member URL)',
           'Total Pageviews',
           'Influenced Room Nights',
           'Economic Impact of IRN',
           'Additional Person Room Nights',
           'Economic Impact of Additional PRN',
           'Visitor Assisted',
           'Satellite Visitor Influence Opportunities',
           'Member Retention Rate',
           'New Members',
           'Member Engagement']

jurisdiction_metrics = ['Definite Bookings',
                       'Booked Room Nights',
                       'Group Business Leads',
                       'Group Lead Room Nights',
                       'Economic Impact of Bookings',
                       'Earned Media',
                       'Unique Page Visitors ( to a jurisdiction page)',
                       'Room Nights booked thru Web',
                       'Economic Impact of RN booked thru web',
                       'Onsite Referrals (clicks to a member listing page)',
                       'Offsite Referrals (clicks to a direct member URL)',
                       'Infl Incremental Room nights',
                       'Economic Impact of Infl room nights**',
                       'Additional Person Room Nights',
                       'Economic Impact of Additional PRN',
                       'New Members']

##############################################################
# Let's create the master frames that will ultimately be the amalgamation of each of the sheets 
# relevant to the respective dataframe

# Master Kpis, these are the 'month' sheets where the main metrics are recorded
master_kpi           = pd.DataFrame()
# The master metrics, these are the 'month metrics' sheets where all of the tracking metrics are stored
master_metrics       = pd.DataFrame()
# The master jurisdiction metrics are found on the same page as the master metrics, just underneath
# In a separate section of the excel document with separate dimensions
master_juris         = pd.DataFrame()

##############################################################
# Let's handle the master KPI frame and out to database 

for sheet in kpi_metrics_sheets:
    try:
        data = pd.read_excel(source_file_path, sheetname = kpi_metrics_sheets[0])
        first_column_title = data.columns[0]
        data = data[data[first_column_title].isin(kpi_metrics)]
        data = data.iloc[:,0:2]
        data = data.rename(columns={first_column_title:'Metrics','Unnamed: 1':'Values'})
        data['Month'] = sheet
    except:
        "unable to read data in pd.read_excel master_kpi"
  
    if data is not None:
        master_kpi = master_kpi.append(data)


##############################################################
# Let's handle the master metrics and master jurisdiction frames

for sheet in metrics_sheets:
    try:
        data = pd.read_excel(source_file_path, sheetname = sheet)
        # This excel sheet is going to have two separate data sets in it due to the structure
        # It's unfortunate, but its time to hack this sheet in half by the 'SALES' column name
        row_slice = int(data[data.iloc[:,0] == 'SALES'].index.tolist()[0])
        #Jurisdiction Metrics
        temp_master_juris = data.ix[row_slice:]
        first_column_title = temp_master_juris.columns[0]
        temp_master_juris = temp_master_juris[temp_master_juris[first_column_title].isin(jurisdiction_metrics)]
        temp_master_juris = temp_master_juris.rename(columns={first_column_title:'Metrics',
                                                              'Unnamed: 1': 'Month Total',
                                                              'Unnamed: 2':'County',
                                                              'Unnamed: 3': 'Monterey',
                                                              'Unnamed: 4': 'Marina',
                                                              'Unnamed: 5': 'Seaside',
                                                              'Unnamed: 6': 'Pacific Grove',
                                                              'Unnamed: 7': 'Salinas',
                                                              'Unnamed: 8': 'Carmel-by-the-Sea'})
        temp_master_juris = temp_master_juris.iloc[:,0:9]
        temp_master_juris['Month'] = sheet.split(' ')[0]
        #Metrics
        temp_master_metrics = data.ix[:row_slice]
        first_column_title = temp_master_metrics.columns[0]
        temp_master_metrics = temp_master_metrics[temp_master_metrics[first_column_title].isin(metrics)]
        temp_master_metrics = temp_master_metrics.iloc[:,0:2]
        temp_master_metrics = temp_master_metrics.rename(columns={first_column_title:'Metrics','Unnamed: 1':'Values'})
        temp_master_metrics['Month'] = sheet.split(' ')[0]

    except:
        "unable to read data in pd.read_excel master_metrics"
        
    if temp_master_juris is not None:
        master_juris = master_juris.append(temp_master_juris)

    if temp_master_metrics is not None:
        master_metrics = master_metrics.append(temp_master_metrics)


master_juris.to_csv(target_jurisdiction_file_path, index = False)
master_metrics.to_csv(target_metrics_file_path, index = False)
master_kpi.to_csv(target_kpi_file_path, index=False)


,Metrics,Month Total,County,Monterey,Marina,Seaside,Pacific Grove,Salinas,Carmel-by-the-Sea,Month
39,Definite Bookings,18,1,15,0,1,0,0,1,July
40,Booked Room Nights,2849,157,2442,0,180,0,0,70,July
41,Group Business Leads,62,28,58,4,14,11,2,6,July
42,Group Lead Room Nights,17524,3802,17239,409,2631,2283,315,482,July
43,Economic Impact of Bookings,4.30306e+06,207223,3.99601e+06,0,61287.5,0,0,38535.4,July
46,Earned Media,996752,716510,461807,222796,82316,29799,427.35,682030,July
47,Unique Page Visitors ( to a jurisdiction page),19431,9774,2361,447,571,1501,689,4088,July
48,Room Nights booked thru Web,193,10,93,8,22,23,12,25,July
49,Economic Impact of RN booked thru web,64269,3330,30969,2664,7326,7659,3996,8325,July
50,Onsite Referrals (clicks to a member listing p...,6510,2278,2058,157,56,618,218,1024,July
